In [48]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision,context_recall

In [20]:
# load DB dataset
file_path = "/Users/masang/Desktop/aiffel/mini-aiffelton/tistory_texts.txt"
loader = TextLoader(file_path)
text = loader.load()

In [21]:
# split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=30,
    length_function=len,
)
docs_after_split = text_splitter.split_documents(text)

print(docs_after_split[0])

page_content='https://level.goorm.io/exam/195698/연합/quiz/1
백준 바이러스와 유사한 문제이다. 그래프로 연결할 때, 몇 개의 집합이 있는가 묻는 문제이다.
DFS와 BFS 방식 두 가지으로 모두 해결했지만, 여기에는 조금 다른 방법을 적어본다.
3주차에서 '탐색'이라는 카테고리에서 DFS와 BFS 응용을 했으니, 4주차에서는 진짜 '그래프'로 풀어보고자 했다.
구름의 출제 의도가 그것이 아닐까? 라는 생각이 들어 다른 그래프 알고리즘을 적용해보았다.
그게 아니라면 DFS와 BFS를 굳이 2주에 걸쳐 할 필요가 없으니 말이다.
Union-Find, 다른 말로는 DSU(Disjoint Set Union)으로 부르는 알고리즘이다.
각 노드(node)를 돌면서, 연결된 노드들 사이에서 root를 찾아 저장한다.
모든 노드를 돌면 각 노드의 저장값은, 루트 노드만이 존재하게 된다.
중복을 제거하면 루트 노드들의 값이 1개씩만 존재할 것이고, 이는 곧 집합의 수와 같다.' metadata={'source': '/Users/masang/Desktop/aiffel/mini-aiffelton/tistory_texts.txt'}


In [ ]:
# text embedding to store in FAISS database
embedding_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-sts',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = FAISS.from_documents(docs_after_split,
                                   embedding = embedding_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )

In [26]:
# save DB in local
vectorstore.save_local("faiss_db")

In [56]:
# retrieve context from db

db = FAISS.load_local('faiss_db', 
                      embedding_model, 
                      allow_dangerous_deserialization=True) # load db

retriever = db.as_retriever(search_kwargs={'k': 1})

query = 'MySQL에서 데이터베이스를 생성하는 방법은?'
context = retriever.get_relevant_documents(query)

In [58]:
model_id = "gogamza/kobart-base-v2"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드
generator = pipeline("text-generation", 
                     model=model_id, 
                     tokenizer=tokenizer)

# 입력 구성
prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

# 입력 토큰 길이 확인
input_tokens = tokenizer(prompt, return_tensors="pt").input_ids
input_length = input_tokens.shape[1]  # 입력 길이

# 답변 생성 (max_new_tokens 사용)
output = generator(prompt, max_new_tokens=50, num_return_sequences=1)

# 출력 결과
print(output[0]["generated_text"])

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
Some weights of BartForCausalLM were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['lm_head.weight', 'model.decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


Context: [Document(id='f22ff308-dedb-4920-bceb-862d6e46c325', metadata={'source': '/Users/masang/Desktop/aiffel/mini-aiffelton/tistory_texts.txt'}, page_content='use_db_name에 접근할 데이터베이스 이름을 적으면 된다.\n이때 MySQL Workbench 좌측창에서 더블 클릭으로 접근해도 상관없다.\n현재 선택한 데이터베이스를 확인한다.\n데이터베이스 내부에 존재하는 테이블 목록을 보여준다.\n특정 테이블의 정보를 확인한다.\n특정 테이블의 Field, Type, Null, Key, Default, Extra 값들을 보여준다.\ntable_name에 보고 싶은 테이블 이름을 적으면 된다.\n테이블을 삭제할 때 사용한다. del_table_name에 삭제할 테이블 이름을 적으면 된다.\n테이블을 생성하는 방법이다.\n위에서 언급한 것처럼 테이블에는Field, Type, Null, Key, Default, Extra 값이 존재한다.\n그에 따라 테이블을 생성할 때는 값을 명시해줘야 한다.\n가장 간단한 형태로 테이블을 생성한 예시이다.\ncats라는 이름으로 테이블을 생성하고 name, breed, age의 Field를 생성했다.')]
Question: MySQL에서 데이터베이스를 생성하는 방법은?
Answer:적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.
적이었다.



In [53]:
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="refine", 
                                 retriever=retriever, 
                                 return_source_documents=False)

result = qa.run({"input": query})
print(result["result"])

ValueError: Missing some input keys: {'query'}

In [ ]:
# evaluate RAG with metrics of RAGAS
data = {
    "user_input": [query],  
    "response": ["한국의 수도는 도쿄입니다."],  # ❌ 틀린 정보
    "retrieved_contexts": [contexts], 
    "reference": ["서울은 대한민국의 수도이다."]  # ✅ 정답
}

dataset = Dataset.from_dict(data)

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

results = evaluate(dataset, metrics)

df = results.to_pandas()
print(df)

Evaluating: 100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


       user_input                                 retrieved_contexts  \
0  한국의 수도는 어디인가요?  [서울은 대한민국의 수도이며, 정치, 경제, 문화의 중심지이다., 도쿄는 일본의 수...   

         response        reference  faithfulness  answer_relevancy  \
0  한국의 수도는 도쿄입니다.  서울은 대한민국의 수도이다.           0.0          0.851921   

   context_precision  context_recall  
0           0.833333             1.0  
